Test case for handling overlap mesages

In [2]:
import numpy as np

# Handle overlap

In [ ]:
## function copied from v3.3 

verbose=True
def _update_feed_handle_overlap(target_feed, incoming_ids, incoming_shares):
    """
    - target_feed (tuple of lists): (message_ids, no_shares, ages), represents an agent's news feed
    """
    # new_message_age = 0
    # check overlap with existing messages
    # if message exists, reset age
    # for the rest of the messages, increment age

    messages, no_shares, ages = target_feed

    feed = zip(messages, no_shares, ages)  # list of tuples
    sorted_feed = sorted(list(feed), key=lambda x: x[0])
    new_messages, new_shares, new_ages = [np.array(i) for i in zip(*sorted_feed)]

    sorted_incoming = sorted(
        list(zip(incoming_ids, incoming_shares)), key=lambda x: x[0]
    )
    sorted_incoming_messages, sorted_incoming_shares = [
        np.array(i) for i in zip(*sorted_incoming)
    ]
    # return sorted intersection
    overlap, x_ind, y_ind = np.intersect1d(
        new_messages, sorted_incoming_messages, return_indices=True
    )
    # update shares:
    new_shares[x_ind] += sorted_incoming_shares[y_ind]
    new_ages += np.ones(len(new_ages), dtype=int)
    new_ages[x_ind] = np.zeros(len(y_ind))
    if verbose:
        print("update no_share and age of overlapping messages.. ")

        print(
            f"before:  messages: {new_messages}, shares: {new_shares}, ages: {new_ages}"
        )

    # push new messages into the feed, only append non-overlapping messages
    new_shares = np.insert(new_shares, 0, sorted_incoming_shares[~y_ind])
    new_messages = np.insert(new_messages, 0, sorted_incoming_messages[~y_ind])

    new_ages = np.insert(new_ages, 0, np.zeros(len(sorted_incoming_shares[~y_ind])))

    if verbose:
        print(
            f"updated: messages: {new_messages}, shares: {new_shares}, ages: {new_ages}"
        )
    updated_feed = (list(new_messages), list(new_shares), list(new_ages))
    return updated_feed

## Function

In [3]:
import numpy as np

agent_feeds = dict()

messages = np.array([3, 1, 2, 4])
no_shares = [1, 1, 3, 1]
ages = [1, 1, 1, 1]

agent_feeds["fil"] = (messages, no_shares, ages)

incoming_ids = np.array([100, 2])
incoming_shares = np.array([10, 4])

In [31]:
feed = messages, no_shares, ages

In [32]:
afeed = zip(messages, no_shares, ages)

In [33]:
bfeed = list(zip(feed))

In [34]:
list(bfeed)

[(array([3, 1, 2, 4]),), ([1, 1, 3, 1],), ([1, 1, 1, 1],)]

In [30]:
list(afeed)

[(3, 1, 1), (1, 1, 1), (2, 3, 1), (4, 1, 1)]

In [10]:
def update_feed_handle_overlap(target_id, incoming_ids, incoming_shares):
    messages, no_shares, ages = agent_feeds[target_id]

    feed = zip(messages, no_shares, ages)
    print(feed)
    sorted_feed = sorted(list(feed), key=lambda x: x[0])
    print(sorted_feed)
    new_messages, new_shares, new_ages = [np.array(i) for i in zip(*sorted_feed)]
    print(new_messages, new_shares, new_ages)

    print(incoming_ids, incoming_shares)
    sorted_incoming = sorted(
        list(zip(incoming_ids, incoming_shares)), key=lambda x: x[0]
    )

    sorted_incoming_messages, sorted_incoming_shares = [
        np.array(i) for i in zip(*sorted_incoming)
    ]
    print(sorted_incoming_messages, sorted_incoming_shares)
    # return sorted intersection
    overlap, x_ind, y_ind = np.intersect1d(
        new_messages, sorted_incoming_messages, return_indices=True
    )

    # update shares:
    print("update no_share and age of overlapping messages.. ")
    new_shares[x_ind] += sorted_incoming_shares[y_ind]
    new_ages += np.ones(len(new_ages), dtype=int)
    new_ages[x_ind] = np.zeros(len(y_ind))
    print(f"updated:  messages: {new_messages}, shares: {new_shares}, ages: {new_ages}")
    # only append non-overlapping messages
    # non_overlapping_shares = sorted_incoming_shares[~y_ind]
    # non_overlapping_messages = sorted_incoming_messages[~y_ind]
    # push new messages into the feed

    new_shares = np.insert(new_shares, 0, sorted_incoming_shares[~y_ind])
    new_messages = np.insert(new_messages, 0, sorted_incoming_messages[~y_ind])

    new_ages = np.insert(new_ages, 0, np.zeros(len(sorted_incoming_shares[~y_ind])))
    print(f"updated: messages: {new_messages}, shares: {new_shares}, ages: {new_ages}")
    new_feed = (new_messages, new_shares, new_ages)
    return new_feed

### Test function

In [17]:
# incoming messages, if not overlapping are added at the beginning of the list
# if a message overlaps, its popularity is updated (+=new popularity), and age is reset to 0
correct_output = ([100, 1, 2, 3, 4], [10, 1, 7, 1, 1], [0, 2, 0, 2, 2])

In [18]:
new_feed = update_feed_handle_overlap("fil", incoming_ids, incoming_shares)

[(1, 1, 1), (2, 3, 1), (3, 1, 1), (4, 1, 1)]
[1 2 3 4] [1 3 1 1] [1 1 1 1]
[100   2] [10  4]
[  2 100] [ 4 10]
update no_share and age of overlapping messages.. 
updated:  messages: [1 2 3 4], shares: [1 7 1 1], ages: [2 0 2 2]
updated: messages: [100   1   2   3   4], shares: [10  1  7  1  1], ages: [0 2 0 2 2]


In [19]:
for idx, alist in enumerate(correct_output):
    assert (new_feed[idx] == alist).all()

## Debug

In [242]:
messages = np.array([3, 1, 2, 4])
no_shares = [1, 1, 3, 1]
ages = [1, 1, 1, 1]
incoming = np.array([100, 2])
incoming_shares = np.array([10, 4])
# correct_output = ([1,2,3,4,100], [1, 3, 1,10], [1, 0, 1, 1,0])

In [243]:
feed = zip(messages, no_shares, ages)
print(feed)
sorted_feed = sorted(list(feed), key=lambda x: x[0])
print(sorted_feed)
new_messages, new_shares, new_ages = [np.array(i) for i in zip(*sorted_feed)]
print(new_messages, new_shares, new_ages)

print(incoming, incoming_shares)
sorted_incoming = sorted(list(zip(incoming, incoming_shares)), key=lambda x: x[0])

sorted_incoming_messages, sorted_incoming_shares = [
    np.array(i) for i in zip(*sorted_incoming)
]
print(sorted_incoming_messages, sorted_incoming_shares)
# return sorted intersection
overlap, x_ind, y_ind = np.intersect1d(
    new_messages, sorted_incoming_messages, return_indices=True
)

sorted_incoming_messages, sorted_incoming_shares
# if message exists, reset age
# for the rest of the messages, increment age
overlap, x_ind, y_ind

[(1, 1, 1), (2, 3, 1), (3, 1, 1), (4, 1, 1)]
[1 2 3 4] [1 3 1 1] [1 1 1 1]
[100   2] [10  4]
[  2 100] [ 4 10]


(array([2]), array([1]), array([0]))

In [244]:
incoming_ages = np.zeros(len(incoming))
print(f"incoming ids : {incoming} --> feed: {messages}")
print(f"no_shares : {incoming_shares} --> feed: {no_shares}")
print(f"incoming age : {incoming_ages} --> feed: {ages}")

incoming ids : [100   2] --> feed: [3 1 2 4]
no_shares : [10  4] --> feed: [1, 1, 3, 1]
incoming age : [0. 0.] --> feed: [1, 1, 1, 1]


In [245]:
new_messages[x_ind]

array([2])

In [246]:
print(f"overlap message between {messages} and {incoming} are: {overlap}")
print(f"sorted messages incoming: {sorted_incoming_messages}")
print(f"no_shares: old: {new_shares[x_ind]}, incoming: {sorted_incoming_shares[y_ind]}")

overlap message between [3 1 2 4] and [100   2] are: [2]
sorted messages incoming: [  2 100]
no_shares: old: [3], incoming: [4]


In [247]:
# # excluding overlapping messages
# mask = np.ones(len(sorted_incoming_messages), dtype=bool)
# mask[[y_ind],]=False
# retain = sorted_incoming_messages[mask]
# retain

In [248]:
new_ages

array([1, 1, 1, 1])

In [249]:
# update shares:
print("update no_share and age of overlapping messages.. ")
new_shares[x_ind] += sorted_incoming_shares[y_ind]
new_ages += np.ones(len(new_ages), dtype=int)
new_ages[x_ind] = np.zeros(len(y_ind))
print(f"updated:  messages: {new_messages}, shares: {new_shares}, ages: {new_ages}")
# only append non-overlapping messages
# non_overlapping_shares = sorted_incoming_shares[~y_ind]
# non_overlapping_messages = sorted_incoming_messages[~y_ind]
new_shares = np.insert(new_shares, 0, sorted_incoming_shares[~y_ind])
new_messages = np.insert(new_messages, 0, sorted_incoming_messages[~y_ind])

new_ages = np.insert(new_ages, 0, np.zeros(len(sorted_incoming_shares[~y_ind])))
print(f"updated: messages: {new_messages}, shares: {new_shares}, ages: {new_ages}")

update no_share and age of overlapping messages.. 
updated:  messages: [1 2 3 4], shares: [1 7 1 1], ages: [2 0 2 2]
updated: messages: [100   1   2   3   4], shares: [10  1  7  1  1], ages: [0 2 0 2 2]


In [173]:
## same as delete [0,2,4]
arr = np.arange(12) + 1

mask = np.ones(len(arr), dtype=bool)

mask[[0, 2, 4]] = False

result = arr[mask]

## Remove messages

In [76]:
ids, shares, ages = new_feed
ids, shares, ages

(array([100,   1,   2,   3,   4]),
 array([10,  1,  7,  1,  1]),
 array([0, 2, 0, 2, 2]))

In [77]:
sorted_by_age = sorted(zip(ids, shares, ages), key=lambda x: x[2])
sorted_by_age

[(100, 10, 0), (2, 7, 0), (1, 1, 2), (3, 1, 2), (4, 1, 2)]

In [78]:
sorted_messages, sorted_shares, sorted_ages = [np.array(i) for i in zip(*sorted_by_age)]
sorted_messages, sorted_shares, sorted_ages

(array([100,   2,   1,   3,   4]),
 array([10,  7,  1,  1,  1]),
 array([0, 0, 2, 2, 2]))

# Rank messages

In [36]:
b = np.array([2, 1])
c = b**0.9
c

array([1.86606598, 1.        ])

In [37]:
new_feed

(array([100,   1,   2,   3,   4]),
 array([10,  1,  7,  1,  1]),
 array([0, 2, 0, 2, 2]))

In [47]:
aging_lambda = 0.9
new_feed = update_feed_handle_overlap("fil", incoming_ids, incoming_shares)
ids, shares, ages = new_feed

relative_shares = shares / np.sum(shares)

recency = ages**aging_lambda

engagement = [0.5, 0, 0.1, 0.5, 0.5]

print("\nrelative shares: ", relative_shares)
assert sum(relative_shares) == 1
print("recency: ", recency)
print("engagement: ", engagement)

ranking = 1 / 3 * np.sum([recency, relative_shares, engagement], axis=0)
ranking

[(1, 1, 1), (2, 3, 1), (3, 1, 1), (4, 1, 1)]
[1 2 3 4] [1 3 1 1] [1 1 1 1]
[100   2] [10  4]
[  2 100] [ 4 10]
update no_share and age of overlapping messages.. 
updated:  messages: [1 2 3 4], shares: [1 7 1 1], ages: [2 0 2 2]
updated: messages: [100   1   2   3   4], shares: [10  1  7  1  1], ages: [0 2 0 2 2]

relative shares:  [0.5  0.05 0.35 0.05 0.05]
recency:  [0.         1.86606598 0.         1.86606598 1.86606598]
engagement:  [0.5, 0, 0.1, 0.5, 0.5]


array([0.33333333, 0.63868866, 0.15      , 0.80535533, 0.80535533])

In [63]:
norm_ranking = normalize(ranking)
assert sum(norm_ranking) == 1
norm_ranking

array([0.12197803, 0.23371795, 0.05489011, 0.29470696, 0.29470696])

In [68]:
aging_lambda = 0.9
new_feed = update_feed_handle_overlap("fil", incoming_ids, incoming_shares)
ids, shares, ages = new_feed

relative_shares = shares / np.sum(shares)

recency = ages**aging_lambda

engagement = np.array([0.5, 0, 0.1, 0.5, 0.5])

print("\nrelative shares: ", relative_shares)
assert sum(relative_shares) == 1
print("recency: ", recency)
print("engagement: ", engagement)

ranking_ = np.sum(
    [1 / 3 * recency, 1 / 3 * relative_shares, 1 / 3 * engagement], axis=0
)

[(1, 1, 1), (2, 3, 1), (3, 1, 1), (4, 1, 1)]
[1 2 3 4] [1 3 1 1] [1 1 1 1]
[100   2] [10  4]
[  2 100] [ 4 10]
update no_share and age of overlapping messages.. 
updated:  messages: [1 2 3 4], shares: [1 7 1 1], ages: [2 0 2 2]
updated: messages: [100   1   2   3   4], shares: [10  1  7  1  1], ages: [0 2 0 2 2]

relative shares:  [0.5  0.05 0.35 0.05 0.05]
recency:  [0.         1.86606598 0.         1.86606598 1.86606598]
engagement:  [0.5 0.  0.1 0.5 0.5]


In [70]:
ranking

array([0.33333333, 0.63868866, 0.15      , 0.80535533, 0.80535533])

In [71]:
ranking_

array([0.33333333, 0.63868866, 0.15      , 0.80535533, 0.80535533])

In [74]:
np.equal(ranking, ranking_)

array([ True,  True, False, False, False])

In [73]:
assert (ranking == ranking_).all()

AssertionError: 

In [56]:
def normalize_v(v):
    normalized_v = v / np.sqrt(np.sum(v**2))
    return normalized_v

In [60]:
def normalize(v):
    norm = np.linalg.norm(v, ord=1)
    if norm == 0:
        return v
    return v / norm

In [61]:
norm_ranking = normalize(ranking)
assert sum(norm_ranking) == 1

array([0.33333333, 0.63868866, 0.15      , 0.80535533, 0.80535533])

In [58]:
sum(norm_ranking)

2.01529355175264

In [45]:
sum(norm_ranking)

2.01529355175264